# Scraper tasa de cambio Dólar

In [50]:
import requests
from bs4 import BeautifulSoup
# Para saber el encoding utilizado por el servidor
import chardet

# Leer la página del URL y devolver un objeto BeautifulSoup

def devolver_soup(url):
    # Eliminamos el '/' del final, si existe
    # Da error 403 si se deja
    if url[len(url)-1] == '/':
        url = url[:len(url)-1]
    
    # Obtenemos la página
    try:
        resp = requests.get(url)
    except Exception as e:
        print('Error obteniendo la página. Excepción ',e)
        return url
    
    if resp.status_code != 200:
        print('Error obteniendo la página. Status Code', resp.status_code)
        return url
    
    # Para saber el encoding utilizado por el servidor
    # print(chardet.detect(resp.content))
    
    # Para especificar el tipo de encoding y 
    # obtener correctamente los caracteres acentuados
    resp.encoding = "utf-8"
    contenido = resp.content
    soup = BeautifulSoup(contenido, 'lxml')    

    return soup

In [51]:
import re

# Función general que extrae datos numéricos 
# de una cadena de caracteres. Devuelve un float

# Elimina los "'" y los puntos de mil
# Reemplaza las "," decimales por "."
# Extrae una cifra y la devuelve como float
def extraer_numero(cadena):
    
    result = cadena.replace("'", '').replace('.', '').replace(',', '.')
    
    result = re.findall(r'-?\d+\.?\d*', result)[0]
    
    return float(result)
 

In [52]:
# Extraemos los datos del dólar

URL='https://monitordolarvenezuela.com/historial#2a'

soup = devolver_soup(URL)

# Extraemos la tabla y sus datos
datos = soup.find('table', attrs={'class':'zui-table'}).find_all('tr')

In [53]:
# Extraemos la información de cada línea de la tabla
result = []

for dato in datos:

  dic = {}
  
  tasas = dato.find_all('td')

  if tasas:

    dic['fecha'] = tasas[0].text.strip()
    dic['alta'] = extraer_numero(tasas[1].text.strip())
    dic['baja'] = extraer_numero(tasas[2].text.strip())

    result.append(dic)
    

In [54]:
# Guardamos el resultado en un archivo CSV
import pandas as pd
from pathlib import Path

BASE_DIR = Path.cwd()
FILE= 'datos/in/tasa_dolar.csv'

In [55]:
df = pd.DataFrame(result)
df


,fecha,alta,baja
0,02-02-2022,4.69,4.70
1,01-02-2022,4.69,4.74
2,31-01-2022,4.73,4.75
3,30-01-2022,4.73,4.74
4,29-01-2022,4.73,4.75
...,...,...,...
806,19-11-2019,25931.00,33034.00
807,18-11-2019,29027.00,30849.00
808,17-11-2019,28277.00,29509.00
809,16-11-2019,28430.00,29229.00


In [57]:
df.to_csv(f"{BASE_DIR / FILE}", sep=';', index=False)